### Load the dataset

In [6]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

### Create embedding for the document 

In [7]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

v = model.encode('I just discovered the course. Can I still join?')
len(v)

384

### Connect to elasticsearch

In [8]:
from elasticsearch import Elasticsearch

def connect_to_es():
    for _ in range(10):  # Retry up to 10 times
        try:
            es = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'DkIedPPSCb'))
            if es.ping():
                return es
        except Exception as e:
            print(f"Connection failed, retrying... ({e})")
            time.sleep(10)
    raise Exception("Failed to connect to Elasticsearch after several retries")
    
es_client = connect_to_es()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

C:\Users\padma\AppData\Local\Temp\ipykernel_89116\2223916736.py:53: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### create index at elasticsearch

In [9]:
from tqdm.auto import tqdm

def prepare_index(documents):
    for doc in tqdm(documents):
        question = doc['question']
        text = doc['text']
        qt = question + ' ' + text

        doc['question_vector'] = model.encode(question)
        doc['text_vector'] = model.encode(text)
        doc['question_text_vector'] = model.encode(qt)

    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)
        
prepare_index(documents)

  0%|          | 0/948 [00:00<?, ?it/s]

  0%|          | 0/948 [00:00<?, ?it/s]

In [10]:
query = 'I just discovered the course. Can I still join it?'
v_q = model.encode(query)

#### search the elasticsearch for the query

In [11]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [12]:
def question_vector_knn(q,search_column):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn(search_column, v_q, course)

### load the ground truth data

In [13]:
import pandas as pd

df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

#### evaluate the search results against the ground truth

In [14]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function, search_column=None):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q,search_column)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [15]:
evaluate(ground_truth, question_vector_knn,search_column='question_vector')

  0%|          | 0/4627 [00:00<?, ?it/s]

C:\Users\padma\AppData\Local\Temp\ipykernel_89116\2120673300.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_results = es_client.search(


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

ES text only: 0.7395720769397017, 0.6032418413658963

In [22]:
evaluate(ground_truth, question_vector_knn,search_column='text_vector')

  0%|          | 0/4627 [00:00<?, ?it/s]

C:\Users\padma\AppData\Local\Temp\ipykernel_36848\2120673300.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_results = es_client.search(


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [48]:
evaluate(ground_truth, question_vector_knn, search_column='question_text_vector')

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

In [29]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [35]:
def evaluate(ground_truth, search_function,search_col):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q,search_col)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

def vector_combined_knn(q,search_col):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn, '')

  0%|          | 0/4627 [00:00<?, ?it/s]

C:\Users\padma\AppData\Local\Temp\ipykernel_36848\210236714.py:38: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_results = es_client.search(


{'hit_rate': 0.9023125135076724, 'mrr': 0.804480945176861}